In [1]:
#import libraries
from collections import defaultdict , Counter ,OrderedDict, deque
import json
import gzip
import networkx as nx
from tqdm import tqdm_notebook as tqdm
from collections import deque, defaultdict
from tqdm import tqdm_notebook as tqdm
import gzip
import functools
import json
from functions_used_1 import *
import time

## Research question 1

From [Wikicat hyperlink graph](https://drive.google.com/file/d/1ghPJ4g6XMCUDFQ2JPqAVveLyytG8gBfL/view), it is downloaded a file which contains a reduced version of the Wikipedia graph of the one found SNAP. In this file, every row is equivalent to a node and the two elements are the nodes source (first) and destination(second). Using this information, it will be built the graph $G=(V,E)$, where $V$ is the set of articles and $E$ the hyperlinks among them. In addition, it will be provided the following information about the graph:

* The number of nodes
* The number of edges
* The graph is directed or undirected
* The graph is dense or sparse
* The average node degree


First, it is computed the number of nodes of $G$.

In [2]:
#read reduced Wikipedia graph file
op = open('wiki-topcats-reduced.txt', 'r', encoding="utf-8")
GVE= (op.read())
op.close()

In [3]:
#set with all the nodes
V=set(GVE.split()) 
print("The number of nodes is "+str(len(V))+".")

The number of nodes is 461193.


Below, it is computed the number of edges:

In [4]:
a=GVE.split()
E=([a[i]+" "+a[i+1] for i in range(0,len(a),2)])
print("The number of edges is " +str(len(E))+".")

The number of edges is 2645247.


Then, the graph $G$ is composed by the sets $V$ and $E$ already computed.

In order to see if $G$ is directed or undirected, it is computed the adjacency list of the graph, $Adj$. The adjacency list by definition is an array(in our case it will be created a dictionary) of $|V|$ lists that for each vertex $u\in V$, $Adj[u]$ contains all the vertices $v$ such that there is an edge $(u,v)\in E$.

In [5]:
#creation of the adjacency list using default dictionary
m=defaultdict(list)
for i,j in zip(*[iter(GVE.split())]*2):
       m[int(i)]+=[int(j)]

Below, it is computed the sum of the lengths of all adjacency lists because the graph is directed if the sum is equal to $|E|$ and undirected if the sum is equal to $2|E|$.


In [6]:
#dictionary :key:value->node:length of adjacency list
length_adj= {k:len(v) for k, v in m.items()}
#sum of all the lengths of the lists
sum(list(length_adj.values())) 

2645247

As the sum of the lenghts of all adjacency lists is equal to $|E|$, the graph is directed.

It is possible to know if the graph is dense or sparse comparing the values of $|V|^2$ and $|E|$. Below are computed these two values:

In [7]:
print("The cardinality of the set of edges is "+str(len(E))+".")
print("The cardinality of the set of nodes power 2 is "+str(len(V)**2)+".")

The cardinality of the set of edges is 2645247.
The cardinality of the set of nodes power 2 is 212698983249.


Due to the fact that $|E|<<|V|^2$, the graph $G$ is sparse.

Below, it is computed the average node degree which is the average number of edges that a node is incident with. This is obtained in this case with the formula $\frac{|E|}{|V|}$. 

In [8]:
print('The average node degree is '+str(len(E)/len(V))+".")

The average node degree is 5.735661642739591.


## Research question 2

In this section, it is used the file <code>wiki-topcats-categories.txt.gz</code> where the articles are grouped by categories. 

In [9]:
#reading categories files
with gzip.open('wiki-topcats-categories.txt.gz') as f:
    categories_file = f.read()

Then, it is created a dictionary where the keys are each one of the categories and the values a list with the articles belonging to the category indicated by the key.

In [10]:
#dictionary with all categories and all the articles that belong 
category_dict=defaultdict(list)
for i in categories_file.split():
    if i.isdigit()==False: 
        key=str(i)[11:-2]
    else:
        category_dict[key]+=[int(i)]

It will be just considered the categories that has a number of articles greater than 3500.

In [11]:
#dictionary with categories with more than 3500 elements
category_dict_1= {k:(v) for k, v in category_dict.items() if len(v)>3500}

In [12]:
print('In fact, it is considered '+str(len(category_dict_1))+" categories." )

In fact, it is considered 35 categories.


Then, from the previous dictionary it is just taken into account the elements which are in the reduced version of the graph provided for the homework.

In [13]:
#creation of dictionary with categories with more than 3500 elements and whose elements are in the reduced graph
A=set(list(map(int,list(V))))
category_dict_2={k:A.intersection(v) for k,v  in category_dict_1.items()}

Then, given an input category $C_0$, with the aim of ordering the categories it is built a block ranking: 
\begin{equation}
block_{RANKING} =\begin{bmatrix} C_0 \\ C_1 \\ \dots \\ C_c\\ \end{bmatrix}
\end{equation}
where each $C_i$ is a category which contains a list of nodes.

The first category of the rank $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

\begin{equation}
distance(C_0, C_i) = median(ShortestPath(C_0, C_i))
\end{equation}

The lower is the distance from $C_0$, the higher is $C_i$ in the rank. $ShortestPath(C_0, C_i)$ is the set of all the possible shortest paths between the nodes of $C_0$ and $C_i$. For this case, it is considered the length of a path as the sum of the edges it is composed by.

In order to compute the shortest path, it is used the Breadth-First Search (BFS) due to the fact that the solutions obtained with the BFS are optimal because of the theorem of Correctness of Breadth-First search. As $|E|$ and $|V|$ are very high, in order to compute the median of the set of shortest paths between the elements of categories $C_i$ and $C_0$, it will be used the following stategy:

* It is calculated the distances from all the articles in the category $C_0$ in the graph to all other nodes of the graph using BFS and they are stored in a dictionary whose keys are the destination nodes. 

* The values are grouped in each category.
* To compute the median for each category, it is taken into account that if all the nodes are connected there will be $|C_0|\times|C_i|$ distance values('Ideal cardinality'). When this is not the case, it means that there are some nodes which are unconnected and thereby unreachable. In this situation, the distance is considered as infinite, that for practical reasons in this report will be symbolised with the value 9999999999 afterwards. However, these values are not computed using the BFS, since the algorithm cannot reach those nodes. Nevertheless, as we know what would be the  'Ideal cardinality' we do not need to store them, just consider as they were there, since the median is the middle elements when the cardinality is an odd number and the mean of the n and n+1 element when the number of elements is odd. Therefore, for each category $C_i$, the difference in number of elements between the cardinality obtained and 'Ideal cardinality' are considered as infinite. 

The input category $C_0$ considered is "Years_of unknown". As it was said before first it is computed the distances from all the nodes in $C_0$ to all the other using Breadth First Search.

In [17]:
#creation of dictionary whose keys are source vectors and the values stored are all distances 
dfault=defaultdict(list)
kl=deque([])
for i in tqdm(category_dict_2["Year_of_birth_unknown"]):
    for k,j in (BFS_13(m,i).items()):
        dfault[int(k)]+=[int(j)]
        

The dictionary with the distances was saved in order to not run the whole search again.

In [19]:
#the dictionary with all distances are stored in a txt.file
with open('BFS_Result.txt', 'w') as f: 
    json.dump(dfault, f)

In [14]:
#the dictionary with all distances is loaded in a variable
with open('BFS_Result.txt') as f: 
    bfsresult = json.load(f)


Then the block ranking is computed with the stretegy explained before: 

In [15]:
block_ranking=block_ranking_creation(category_dict_2,bfsresult, 'Year_of_birth_unknown')

It is shown the block ranking with the corresponding value of the median for its category. The value 999999999999999999 corresponds to infinte and it means most of the nodes of that category was unconnected with the input category.

In [16]:
block_ranking

OrderedDict([('Year_of_birth_unknown', -1),
             ('English-language_films', 6.0),
             ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
              7.0),
             ('English_television_actors', 7.0),
             ('British_films', 7.0),
             ('American_films', 7.0),
             ('American_Jews', 7.0),
             ('American_television_actors', 7.0),
             ('American_film_actors', 7.0),
             ('Black-and-white_films', 7.0),
             ('Indian_films', 8.0),
             ('Rivers_of_Romania', 8.0),
             ('English-language_albums', 8.0),
             ('People_from_New_York_City', 8.0),
             ('Article_Feedback_Pilot', 8.0),
             ('Debut_albums', 9.0),
             ('English_footballers', '999999999999999999'),
             ('The_Football_League_players', '999999999999999999'),
             ('Association_football_forwards', '999999999999999999'),
             ('Association_football_goalkeepers', '9

Once the $block_{RANKING}$ vector is obtained , the nodes in each category are sorted. The way this is done is explained below.
Suppose the categories order, given from the previous point, are $C_0, C_1 \ldots, C_n$.

* It is computed the subgraph induced by $C_0$. For each node compute the sum of the weigths of the in-edges, that is to say,
\begin{equation}
score_{article_i} = \sum_{j \in in-edges(article_i)} w_j
\end{equation}
* The graph is extended to the nodes that belong to $C_1$. Thus, for each article in $C_1$ it is computed the score as before. Note that the in-edges coming from the previous category, $C_0$, have as weights the score of the node that sends the edge.

* The previous step is repeated up to the last category of the ranking. 

Before, doing the process described above, the articles which are in several categories are just considered, among the categories it belongs to, to the closest to the input category. This is done below:

In [17]:
#block ranking with just names(no distances)
block_ranking_names=block_ranking.keys()
#categories without repeated elements
filter_categories=categories_clean(block_ranking,category_dict_2)

Then, it is created a function which creates a list whose elements are the nodes considered in each iteration.

In [19]:
# creation of lists with the nodes of the subgraph for each iteration 
steps=step_creation(filter_categories)

It is created a function using the library <code>networkx</code> to order the elements of each category as it was described before. The function returns a list of dictionaries, where each list corresponds to a category of the block ranking.

In [20]:
o=articles_order(m,filter_categories,steps)

In [30]:
#merge all the ranks
D=[]
for i in o:
    # all articles ordered are included in a list
    D.extend(OrderedDict(sorted(i.items(), key=lambda t: int(t[1]) ,reverse=True)).keys())

The first 5000 elements of the rank are shown below.

In [35]:
print(D[:5000])

[62684, 170163, 1656777, 1656780, 169696, 1656794, 170578, 1342864, 1343014, 1656778, 666855, 1342960, 1203095, 1203235, 1203496, 1779656, 174582, 1109348, 1109485, 159606, 159730, 159920, 1766063, 64632, 1344701, 34422, 1443739, 166284, 168001, 168145, 168251, 168258, 185120, 170158, 1203101, 170969, 170970, 170971, 170972, 170973, 1122762, 174427, 174439, 666857, 159750, 1765824, 1765831, 1765837, 62695, 1340874, 1342803, 1343206, 360595, 1344730, 1344821, 1656450, 1656452, 1656453, 1656455, 1443741, 1656779, 1656793, 1345946, 1345947, 167906, 167966, 168100, 168194, 201333, 186174, 186176, 170536, 1203043, 1203247, 171344, 171410, 171660, 958429, 958480, 172050, 1319048, 1122603, 1122605, 156307, 173221, 173671, 1779795, 1779800, 174428, 1190355, 60061, 175366, 1109359, 159614, 159736, 159749, 159753, 159754, 159766, 159914, 159934, 176367, 1765823, 1765832, 1765845, 1765848, 1684172, 324414, 1766449, 1766721, 456791, 1144930, 1341834, 883490, 748777, 1342852, 1342968, 1343059, 1343